# Subject1: process mock dataset


Three cases:

- sender1 will send an email at 22h then 7h
- sender2 will send an email at 22h then 7h one day apart
- sender3 will send an email at 21h then 8h the next day

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import pandas as pd

In [61]:
email_df = pd.read_csv("/content/drive/MyDrive/AI/problems/mock.csv")

In [62]:
email_df

,email_id,sender,receiver,datetime,object
0,1123,sender1@gmail.com,receiver1@gmail.com,01/01/2020 10:55:02,normal email 1
1,1456,sender1@gmail.com,receiver2@gmail.com,01/01/2020 11:55:02,normal email 2
2,1789,sender1@gmail.com,receiver3@gmail.com,01/01/2020 12:55:02,normal email 3
3,1101,sender1@gmail.com,receiver4@gmail.com,01/01/2020 22:55:02,normal email 4
4,1112,sender1@gmail.com,receiver5@gmail.com,02/01/2020 07:55:02,abnormal email 1
5,2123,sender2@gmail.com,receiver1@gmail.com,01/01/2020 10:55:02,normal email 1
6,2456,sender2@gmail.com,receiver2@gmail.com,01/01/2020 11:55:02,normal email 2
7,2789,sender2@gmail.com,receiver3@gmail.com,01/01/2020 12:55:02,normal email 3
8,2101,sender2@gmail.com,receiver4@gmail.com,01/01/2020 21:55:02,normal email 4
9,2112,sender2@gmail.com,receiver5@gmail.com,02/01/2020 06:55:02,abnormal email 1


## add 11h appart flag

In [63]:
# Convert 'datetime' to datetime
email_df['datetime'] = pd.to_datetime(email_df['datetime'], format='%d/%m/%Y %H:%M:%S')

# Create a separate column for the date only (no time)
email_df['date'] = email_df['datetime'].dt.date

# Sort DataFrame by sender and datetime
email_df = email_df.sort_values(['sender', 'datetime'])

# Calculate difference between consecutive emails for each sender
email_df['time_diff'] = email_df.groupby('sender')['datetime'].diff()
email_df['date_diff'] = email_df.groupby('sender')['date'].diff()

# Flag the emails sent less than 11 hours apart on consecutive days
email_df['less_than_11h'] = (email_df['time_diff'].dt.total_seconds() / 3600 < 11) & (email_df['date_diff'].dt.days == 1)


We delete useless features used during the flagging step

In [64]:
del email_df['date_diff']
del email_df['time_diff']

In [65]:
email_df

,email_id,sender,receiver,datetime,object,date,less_than_11h
0,1123,sender1@gmail.com,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False
1,1456,sender1@gmail.com,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False
2,1789,sender1@gmail.com,receiver3@gmail.com,2020-01-01 12:55:02,normal email 3,2020-01-01,False
3,1101,sender1@gmail.com,receiver4@gmail.com,2020-01-01 22:55:02,normal email 4,2020-01-01,False
4,1112,sender1@gmail.com,receiver5@gmail.com,2020-01-02 07:55:02,abnormal email 1,2020-01-02,True
5,2123,sender2@gmail.com,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False
10,3123,sender2@gmail.com,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False
6,2456,sender2@gmail.com,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False
11,3456,sender2@gmail.com,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False
7,2789,sender2@gmail.com,receiver3@gmail.com,2020-01-01 12:55:02,normal email 3,2020-01-01,False


## Add other stats

### sent before 8h and after 21h bools

In [66]:
email_df['is_sent_before_8h'] = email_df['datetime'].dt.hour >= 8
email_df['is_sent_after_21h'] = email_df['datetime'].dt.hour >= 21


In [67]:
email_df

,email_id,sender,receiver,datetime,object,date,less_than_11h,is_sent_before_8h,is_sent_after_21h
0,1123,sender1@gmail.com,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False
1,1456,sender1@gmail.com,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False
2,1789,sender1@gmail.com,receiver3@gmail.com,2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False
3,1101,sender1@gmail.com,receiver4@gmail.com,2020-01-01 22:55:02,normal email 4,2020-01-01,False,True,True
4,1112,sender1@gmail.com,receiver5@gmail.com,2020-01-02 07:55:02,abnormal email 1,2020-01-02,True,False,False
5,2123,sender2@gmail.com,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False
10,3123,sender2@gmail.com,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False
6,2456,sender2@gmail.com,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False
11,3456,sender2@gmail.com,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False
7,2789,sender2@gmail.com,receiver3@gmail.com,2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False


### Sent on saturday and sent on sunday

In [68]:
email_df['day_name'] = email_df.datetime.dt.day_name()

email_df['is_sent_on_sunday'] = email_df.day_name.isin(['Sunday'])
email_df['is_sent_on_saturday'] = email_df.day_name.isin(['Saturday'])

### extract name of sender and receiver

#### replace emails with realistic emails in the mock dataset

In [69]:
email_df['sender'] = email_df['sender'].str.replace('sender1@gmail.com', 'Bryan Temp <bryan.temp@gmail.com>')
email_df['sender'] = email_df['sender'].str.replace('sender2@gmail.com', 'Kelly Kapoor <kelly.kapoor@gmail.com>')
email_df.head()

<ipython-input-69-5fe0025f2bb5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  email_df['sender'] = email_df['sender'].str.replace('sender1@gmail.com', 'Bryan Temp <bryan.temp@gmail.com>')
<ipython-input-69-5fe0025f2bb5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  email_df['sender'] = email_df['sender'].str.replace('sender2@gmail.com', 'Kelly Kapoor <kelly.kapoor@gmail.com>')


,email_id,sender,receiver,datetime,object,date,less_than_11h,is_sent_before_8h,is_sent_after_21h,day_name,is_sent_on_sunday,is_sent_on_saturday
0,1123,Bryan Temp <bryan.temp@gmail.com>,receiver1@gmail.com,2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False
1,1456,Bryan Temp <bryan.temp@gmail.com>,receiver2@gmail.com,2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False
2,1789,Bryan Temp <bryan.temp@gmail.com>,receiver3@gmail.com,2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False
3,1101,Bryan Temp <bryan.temp@gmail.com>,receiver4@gmail.com,2020-01-01 22:55:02,normal email 4,2020-01-01,False,True,True,Wednesday,False,False
4,1112,Bryan Temp <bryan.temp@gmail.com>,receiver5@gmail.com,2020-01-02 07:55:02,abnormal email 1,2020-01-02,True,False,False,Thursday,False,False


In [70]:
email_df['receiver'] = email_df['receiver'].str.replace('receiver1@gmail.com', 'Michael Scott <michael.scott@gmail.com>, Dwight Shrutte <dwight.schrutte@gmail.com>')
email_df['receiver'] = email_df['receiver'].str.replace('receiver2@gmail.com', 'Jim Halpert <jim.halpert@gmail.com>, Maredith Palmer <meredith.palmer@gmail.com>')
email_df['receiver'] = email_df['receiver'].str.replace('receiver3@gmail.com', 'Michael Scott <michael.scott@gmail.com>, Dwight Shrutte <dwight.schrutte@gmail.com>, Jim Halpert <jim.halpert@gmail.com>, Maredith Palmer <meredith.palmer@gmail.com>')
email_df['receiver'] = email_df['receiver'].str.replace('receiver4@gmail.com', 'Michael Scott <michael.scott@gmail.com>')
email_df['receiver'] = email_df['receiver'].str.replace('receiver5@gmail.com', 'Michael Scott <michael.scott@gmail.com>, Dwight Shrutte <dwight.schrutte@gmail.com>, Toby Flenderson <toby.flenderson@gmail.com>')
email_df

<ipython-input-70-26515af209c0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  email_df['receiver'] = email_df['receiver'].str.replace('receiver1@gmail.com', 'Michael Scott <michael.scott@gmail.com>, Dwight Shrutte <dwight.schrutte@gmail.com>')
<ipython-input-70-26515af209c0>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  email_df['receiver'] = email_df['receiver'].str.replace('receiver2@gmail.com', 'Jim Halpert <jim.halpert@gmail.com>, Maredith Palmer <meredith.palmer@gmail.com>')
<ipython-input-70-26515af209c0>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  email_df['receiver'] = email_df['receiver'].str.replace('receiver3@gmail.com', 'Michael Scott <michael.scott@gmail.com>, Dwight Shrutte <dwight.schrutte@gmail.com>, Jim Halpert <jim.halpert@gmail.com>, Maredith Palmer <meredith.palmer@gmail.com>')
<ipython-input-70-26515af209c0

,email_id,sender,receiver,datetime,object,date,less_than_11h,is_sent_before_8h,is_sent_after_21h,day_name,is_sent_on_sunday,is_sent_on_saturday
0,1123,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False
1,1456,Bryan Temp <bryan.temp@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False
2,1789,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False
3,1101,Bryan Temp <bryan.temp@gmail.com>,Michael Scott <michael.scott@gmail.com>,2020-01-01 22:55:02,normal email 4,2020-01-01,False,True,True,Wednesday,False,False
4,1112,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-02 07:55:02,abnormal email 1,2020-01-02,True,False,False,Thursday,False,False
5,2123,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False
10,3123,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False
6,2456,Kelly Kapoor <kelly.kapoor@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False
11,3456,Kelly Kapoor <kelly.kapoor@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False
7,2789,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False


#### extract name and surname function

In [71]:
import re

def extract_names_from_emails(input_string):
    # Step 1: Detect all email addresses using regular expression
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b'
    emails = re.findall(email_regex, input_string)

    name_surname_couples = []
    for email in emails:
        # Step 2: Extract name and surname from email address
        name, surname = email.split('@')[0].split('.')

        # Step 3: Capitalize first letter of name and surname
        name = name.lower().capitalize()
        surname = surname.lower().capitalize()

        # Step 4: Append name and surname to the couples list
        name_surname_couples.append(f"{name} {surname}")

    # Return all name and surname couples separated by ";"
    return ";".join(name_surname_couples)


In [72]:
text = "Hello, my email address is john.doe@example.com. Please contact me if you have any questions. louis.paulet@gmail.com"
email_addresses = extract_names_from_emails(text)
print("Email Addresses:", email_addresses)


Email Addresses: John Doe;Louis Paulet


#### apply extraction of name and surname

In [73]:
email_df['sender_name'] = email_df.sender.apply(extract_names_from_emails)
email_df['receiver_name'] = email_df.receiver.apply(extract_names_from_emails)
email_df

,email_id,sender,receiver,datetime,object,date,less_than_11h,is_sent_before_8h,is_sent_after_21h,day_name,is_sent_on_sunday,is_sent_on_saturday,sender_name,receiver_name
0,1123,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False,Bryan Temp,Michael Scott;Dwight Schrutte
1,1456,Bryan Temp <bryan.temp@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False,Bryan Temp,Jim Halpert;Meredith Palmer
2,1789,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False,Bryan Temp,Michael Scott;Dwight Schrutte;Jim Halpert;Mere...
3,1101,Bryan Temp <bryan.temp@gmail.com>,Michael Scott <michael.scott@gmail.com>,2020-01-01 22:55:02,normal email 4,2020-01-01,False,True,True,Wednesday,False,False,Bryan Temp,Michael Scott
4,1112,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-02 07:55:02,abnormal email 1,2020-01-02,True,False,False,Thursday,False,False,Bryan Temp,Michael Scott;Dwight Schrutte;Toby Flenderson
5,2123,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Michael Scott;Dwight Schrutte
10,3123,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Michael Scott;Dwight Schrutte
6,2456,Kelly Kapoor <kelly.kapoor@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Jim Halpert;Meredith Palmer
11,3456,Kelly Kapoor <kelly.kapoor@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Jim Halpert;Meredith Palmer
7,2789,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Michael Scott;Dwight Schrutte;Jim Halpert;Mere...


# Subject2: Parse weird date

TLDR: doesn't change anything, only a problem in certain times of the year

In [74]:
import pytz
from dateutil import parser

def convert_to_datetime(time_string):
    # Check if string contains 'CEST' or 'CET'
    if "CEST" in time_string:
        # Convert the string into datetime object without timezone
        date_without_tz = parser.parse(time_string.replace("CEST", "").strip())
        # Define CEST timezone
        tz = pytz.timezone('Europe/Paris')
    elif "CET" in time_string:
        # Convert the string into datetime object without timezone
        date_without_tz = parser.parse(time_string.replace("CET", "").strip())
        # Define CET timezone
        tz = pytz.timezone('CET')

    # Add the timezone to the datetime object
    date_with_tz = tz.localize(date_without_tz)

    return date_with_tz


In [75]:
time_string = "06/22/22 5:17:55 PM CET"
result = convert_to_datetime(time_string)
print(result)

time_string = "06/22/22 5:17:55 PM CEST"
result = convert_to_datetime(time_string)
print(result)

2022-06-22 17:17:55+02:00
2022-06-22 17:17:55+02:00


# Export to weird tableau supported format

In [76]:
email_df

,email_id,sender,receiver,datetime,object,date,less_than_11h,is_sent_before_8h,is_sent_after_21h,day_name,is_sent_on_sunday,is_sent_on_saturday,sender_name,receiver_name
0,1123,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False,Bryan Temp,Michael Scott;Dwight Schrutte
1,1456,Bryan Temp <bryan.temp@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False,Bryan Temp,Jim Halpert;Meredith Palmer
2,1789,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False,Bryan Temp,Michael Scott;Dwight Schrutte;Jim Halpert;Mere...
3,1101,Bryan Temp <bryan.temp@gmail.com>,Michael Scott <michael.scott@gmail.com>,2020-01-01 22:55:02,normal email 4,2020-01-01,False,True,True,Wednesday,False,False,Bryan Temp,Michael Scott
4,1112,Bryan Temp <bryan.temp@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-02 07:55:02,abnormal email 1,2020-01-02,True,False,False,Thursday,False,False,Bryan Temp,Michael Scott;Dwight Schrutte;Toby Flenderson
5,2123,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Michael Scott;Dwight Schrutte
10,3123,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 10:55:02,normal email 1,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Michael Scott;Dwight Schrutte
6,2456,Kelly Kapoor <kelly.kapoor@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Jim Halpert;Meredith Palmer
11,3456,Kelly Kapoor <kelly.kapoor@gmail.com>,"Jim Halpert <jim.halpert@gmail.com>, Maredith ...",2020-01-01 11:55:02,normal email 2,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Jim Halpert;Meredith Palmer
7,2789,Kelly Kapoor <kelly.kapoor@gmail.com>,"Michael Scott <michael.scott@gmail.com>, Dwigh...",2020-01-01 12:55:02,normal email 3,2020-01-01,False,True,False,Wednesday,False,False,Kelly Kapoor,Michael Scott;Dwight Schrutte;Jim Halpert;Mere...


In [77]:
import csv
email_df.to_csv('/content/drive/MyDrive/AI/problems/mock_export.csv', sep='|', quoting=csv.QUOTE_ALL)